# Variational Quantum Error Correction with QVECTOR

In this notebook, we will optimize a Quantum Error Correction (QEC) problem. 
Here, our set of states $\{\psi_i\}_{i=1}^N$ are sampled from the Haar distribution (or an approximation thereof) and we optimize the cost 
$$L(\vec{p}, \vec{q}) = \sum_{\vert \psi_i \rangle \in \mathcal{S}} \frac{1}{\vert\mathcal{S}\vert} E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q})$$
where $\vec{p}, \vec{q}$ are parameters of a variational circuit, and $E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q}) = \langle \psi_i \vert \mathcal{V}(\vec{p})^\dagger \mathcal{W}(\vec{q}) \mathcal{V}(\vec{p}) \vert \psi_i \rangle$, with $\mathcal{V}$ the encoding operator and $\mathcal{W}$ the decoding operator.

In [ ]:
import typing as ty
import pennylane as qml
from pennylane import numpy as np
from refoqus import Refoqus

# Initialise the number of qubits that interest us.
k, n, r = 1, 3, 1
nbqbits = n + r

In [ ]:
dataset_size = 200
layers = 1

def random_angles(*shape: int) -> np.ndarray:
    return 2 * np.pi * np.random.rand(*shape)

# Build the dataset here with qml.SimplifiedTwoDesign
# See https://docs.pennylane.ai/en/stable/code/api/pennylane.SimplifiedTwoDesign.html
dataset = [[qml.SimplifiedTwoDesign(initial_layer_weights=random_angles(k), weights=random_angles(layers, k - 1, 2), wires=range(k))] for _ in range(dataset_size)]

In [ ]:
# Get the Hamiltonian of interest
coefficients_cost = [- 1.0 / k for _ in range(k)]
projector = np.zeros((2, 2))
projector[0, 0] = 1
vqec_hamiltonian_term = [qml.Hermitian(projector,wires=i) for i in range(k)]
hamiltonian_of_interest = qml.Hamiltonian(coefficients_cost, vqec_hamiltonian_term)

Next, we define functions to evaluate the true cost during optimization.

In [ ]:
def circuit_construction(
    weights: np.ndarray,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
):
    if parameterised_circuit is None:
        parameterised_circuit = qml.StronglyEntanglingLayers
        
    encoding_shape = parameterised_circuit.shape(layers, n)
    decoding_shape = parameterised_circuit.shape(layers, n + r)
    encoding_size = np.prod(encoding_shape)
    decoding_size = np.prod(decoding_shape)
    encoding_weights = weights[:encoding_size].reshape(encoding_shape)
    decoding_weights = weights[encoding_size:encoding_size + decoding_size].reshape(decoding_shape)
    
    # Apply S         to range(k)     to prepare an approximate 2-design state
    for op in data_circuit:
        qml.apply(op)
    
    # Apply V(p)      to range(n)     to encode the state
    parameterised_circuit(encoding_weights, wires=range(n))
    # Apply W(q)      to range(n + r) to correct potential errors
    parameterised_circuit(decoding_weights, wires=range(n + r))
    # Apply V^{-1}(p) to range(n)     to decode the state
    qml.adjoint(parameterised_circuit(encoding_weights, wires=range(n)))
    
    # Apply S^{-1}    to range(k)     to un-prepare.
    for op in reversed(data_circuit):
        qml.apply(op.inv())
    
def cost_function(
    weights: np.ndarray,
    hamiltonian_terms: qml.operation.Operator,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
):
    circuit_construction(weights, data_circuit, parameterised_circuit)
    return qml.sample(hamiltonian_terms)

In [ ]:
analytic_dev = qml.device("default.qubit", wires=nbqbits, shots=None)

@qml.qnode(analytic_dev)
def cost_analytic_one_circuit(weights, index_datapoint, parameterised_circuit = None):
    circuit_construction(weights, dataset[index_datapoint], parameterised_circuit)
    return qml.expval(hamiltonian_of_interest)

def cost_analytic_alldataset(weights, parameterised_circuit = None):
    cost = 0.0
    for m in range(dataset_size):
        cost += cost_analytic_one_circuit(weights, m, parameterised_circuit)
    cost = 1.0 + cost / dataset_size
    return cost

Now, the ansatz is defined as with StronglyEntanglingLayers. We also sample initial values and the corresponding cost.

Our adaptative optimizer will be Refoqus where we provide the necessary arguments as follows and we perform niter iterations.

In [ ]:
ansatz = qml.StronglyEntanglingLayers

parameter_size: int = np.prod(ansatz.shape(layers, n)) + np.prod(ansatz.shape(layers, n + r))

opt = Refoqus(nbqbits, dataset, vqec_hamiltonian_term, coefficients_cost, param_shape=(parameter_size,), function_cost_term_tosample=cost_function, min_shots=2)
params = random_angles(parameter_size)
niter = 20

cost_refoqus = [cost_analytic_alldataset(params, ansatz)]
shots_refoqus = [0]

for i in range(niter):
    params = opt.step(params)
    cost_refoqus.append(cost_analytic_alldataset(params, ansatz))
    shots_refoqus.append(opt.shots_used)
    print(f"Step {i}: cost = {cost_refoqus[-1]}, shots_used = {shots_refoqus[-1]}")